In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
import os
os.chdir('/content/drive/My Drive/NLP/3/')
os.listdir()

['data',
 'rnn_5_epoch.net',
 '5_epoch_sample.txt',
 'starter_notebook_into_English_training.ipynb',
 '20_epoch_sample.txt',
 'rnn_20_epoch.net',
 'lab3.ipynb']

In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
all_files = []
for dirname, _, filenames in os.walk('./data'):
    for filename in filenames:
        f = os.path.join(dirname, filename)
        all_files.append(f)
        print(f)

        
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

./data/azlyrics_lyrics_19.csv
./data/azlyrics_lyrics_a.csv
./data/azlyrics_lyrics_b.csv
./data/azlyrics_lyrics_c.csv
./data/azlyrics_lyrics_e.csv
./data/azlyrics_lyrics_d.csv
./data/azlyrics_lyrics_f.csv
./data/azlyrics_lyrics_g.csv
./data/azlyrics_lyrics_j.csv
./data/azlyrics_lyrics_h.csv
./data/azlyrics_lyrics_i.csv
./data/azlyrics_lyrics_k.csv
./data/azlyrics_lyrics_l.csv
./data/azlyrics_lyrics_m.csv
./data/azlyrics_lyrics_n.csv
./data/azlyrics_lyrics_o.csv
./data/azlyrics_lyrics_p.csv
./data/azlyrics_lyrics_s.csv
./data/azlyrics_lyrics_q.csv
./data/azlyrics_lyrics_r.csv
./data/azlyrics_lyrics_u.csv
./data/azlyrics_lyrics_w.csv
./data/azlyrics_lyrics_v.csv
./data/azlyrics_lyrics_t.csv
./data/azlyrics_lyrics_z.csv
./data/azlyrics_lyrics_x.csv
./data/azlyrics_lyrics_y.csv


In [5]:
# Let's load the resources
import numpy as np
import torch
import pandas as pd
from torch import nn
import torch.nn.functional as F
import os

# Load in Data

Then, we'll load the song dataset, convert it into a text file and convert it into integers (i.e. one-hot-encoding) for our network to use. 

In [6]:
# TODO: Get all the files, get the column with the lyrics in them and concatenate them
import csv

names = [
         'architects' 
          ,'nirvana' 
          ,'the who'
          ,'pink floyd'
          ,'ac/dc'
          ,'simon & garfunkel'
          ,'billy idol'
          ,'styx'
          ,'the jim caroll band'
          ,'fleetwood mac'
          ,'the drums'
          ,'mazzy star'
          ,'the peggies'
          ,'duran duran'
          ,'the buggles'
          ,'the samples'
          ,'bonnie tyler'
          ,'ezra furman'
          ,'julie london'
          ,'pearl brothers'
          ,'radiohead'
          ,'dave van ronk'
          ,'buffy sainte-marie'
          ,'bee gees'
          ,'the smiths'
          ,'norma tanega'
          ,'the xx'
          ,'still corners'
          ,'kygo'
          ,'dan domer'
          ,'alessia cara'
          ,'julie london'
          ,'bob dylan'
          ,'dexys midnight runners'
          ,'the beatles'
          ,'ella fitzgerald'
          ,'logic'
          ,'audrey hepburn'
          ,'deam martin'
          ,'blossom dearie'
          ]

text = ""
for f in all_files:
    df = pd.read_csv(f,quoting=csv.QUOTE_MINIMAL, delimiter=",", warn_bad_lines=False, error_bad_lines=False)
    df = df[df['ARTIST_NAME'].isin(names)]
    text = text + ("\n".join(list(df["LYRICS"].astype(str)))) + "\n"

print("Number of characters: %d" % len(text))
text[:200]

# df = pd.read_csv('./data/azlyrics_lyrics_b.csv',quoting=csv.QUOTE_MINIMAL, delimiter=",", warn_bad_lines=False, error_bad_lines=False)
# # df = df.loc[df['ARTIST_NAME'] == 'simon & garfunkel']
# df.shape

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (133) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Number of characters: 816019


"\nridin' down the highway, goin' to a show, stop in all the byways, playin' rock 'n' roll, gettin' robbed, gettin' stoned, gettin' beat up, broken boned, gettin' had, gettin' took, i tell you folks, it"


# Tokenization

In the cells, below, I'm creating a couple dictionaries to convert the characters to and from integers. Encoding the characters as integers makes it easier to use as input in the network.


In [7]:
# encode the text and map each character to an integer and vice versa

# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {c: i for i, c in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])

And we can see those same characters from above, encoded as integers.


In [8]:
encoded[:100]

array([35, 10, 21,  6, 21,  0, 43, 24,  6,  1, 20,  0, 24, 16, 37, 41, 24,
       37, 21, 52, 37, 20, 13, 49, 36, 24, 52,  1, 21,  0, 43, 24, 16,  1,
       24, 13, 24, 19, 37,  1, 20, 36, 24, 19, 16,  1, 38, 24, 21,  0, 24,
       13,  9,  9, 24, 16, 37, 41, 24, 12, 49, 20, 13, 49, 19, 36, 24, 38,
        9, 13, 49, 21,  0, 43, 24, 10,  1, 31, 33, 24, 43,  0, 43, 24, 10,
        1,  9,  9, 36, 24, 52, 41, 16, 16, 21,  0, 43, 24, 10,  1])

# Pre-processing the data

Our LSTM expects an input that is one-hot encoded meaning that each character is converted into an integer (via our created dictionary) and then converted into a column vector where only it's corresponding integer index will have the value of 1 and the rest of the vector will be filled with 0's. Since we're one-hot encoding the data, let's make a function to do that!


In [9]:
def one_hot_encode(arr, n_labels):
    # TODO 
    # Initialize the the encoded array
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [10]:
# check that the function works as expected
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


# Making training mini-batches

To train on this data, we also want to create mini-batches for training. Remember, we're training a left-to-right character level language model. This means we want our source to be a sequence of tokens (a window), and our target to be the character we want to predict (a single character).

<br>

In this example, we'll take the encoded characters (passed in as the `arr` parameter) and split them into multiple sequences, given by `batch_size`. Each of our sequences will be `seq_length` long.

### Creating Batches

**1. The first thing we need to do is discard some of the text so we only have completely full mini-batches. **

Each batch contains $N \times M$ characters, where $N$ is the batch size (the number of sequences in a batch) and $M$ is the seq_length or number of time steps in a sequence. Then, to get the total number of batches, $K$, that we can make from the array `arr`, you divide the length of `arr` by the number of characters per batch. Once you know the number of batches, you can get the total number of characters to keep from `arr`, $N * M * K$.

**2. After that, we need to split `arr` into $N$ batches. ** 

You can do this using `arr.reshape(size)` where `size` is a tuple containing the dimensions sizes of the reshaped array. We know we want $N$ sequences in a batch, so let's make that the size of the first dimension. For the second dimension, you can use `-1` as a placeholder in the size, it'll fill up the array with the appropriate data for you. After this, you should have an array that is $N \times (M * K)$.

**3. Now that we have this array, we can iterate through it to get our mini-batches. **

The idea is each batch is a $N \times M$ window on the $N \times (M * K)$ array. For each subsequent batch, the window moves over by `seq_length`. We also want to create both the input and target arrays. Remember that the targets are just the inputs shifted over by one character. The way I like to do this window is use `range` to take steps of size `n_steps` from $0$ to `arr.shape[1]`, the total number of tokens in each sequence. That way, the integers you get from `range` always point to the start of a batch, and each window is `seq_length` wide.

In [11]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    batch_size_total = batch_size * seq_length
    # total number of batches we can make, // integer division, round down
    n_batches = len(arr)//batch_size_total
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size_total]
    # Reshape into batch_size rows, n. of first row is the batch size, the other lenght is inferred
    arr = arr.reshape((batch_size, -1))
    
    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        
        # The targets, shifted by one
        y = np.zeros((x.shape), dtype=x.dtype)
        try:
          y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
          y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]

        
        yield x, y 
        
#when we call get batches we are going 
#to create a generator that iteratest through our array and returns x, y with yield command



# Test Your Implementation

Now let's make some data sets and we can check out what's going on as we batch data. Here, as an example, lets use a batch size of 8 and 50 sequence steps.


In [12]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)
# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[35 10 21  6 21  0 43 24  6  1]
 [ 6 41 13 10 41 19 16 43 24 16]
 [52  1 50 36 24 21 43  3 24  0]
 [10 41  3 41  3 12 41 10 24  3]
 [33 41 24 21 16 43 19 24 52  1]
 [ 0 13 24  3 21  0  6 24 21 26]
 [10 13 10 41 36 24 16 37 21 19]
 [52  1  0  0 13 24  6  1 24 20]]

y
 [[10 21  6 21  0 43 24  6  1 20]
 [41 13 10 41 19 16 43 24 16  1]
 [ 1 50 36 24 21 43  3 24  0  1]
 [41  3 41  3 12 41 10 24  3 13]
 [41 24 21 16 43 19 24 52  1  0]
 [13 24  3 21  0  6 24 21 26 24]
 [13 10 41 36 24 16 37 21 19 24]
 [ 1  0  0 13 24  6  1 24 20 21]]



# Model Structure

In `__init__` the suggested structure is as follows:

- Create and store the necessary dictionaries
- Define an LSTM layer that takes as params: an input size (the number of characters), a hidden layer size `n_hidden`, a number of layers `n_layers`, a dropout probability `drop_prob`, and a `batch_first` boolean (True, since we are batching)
- Define a dropout layer with `dropout_prob`
- Define a fully-connected layer with params: input size `n_hidden` and output size (the number of characters)
- Finally, initialize the weights (again, this has been given)

Note that some parameters have been named and given in the `__init__` function, and we use them and store them by doing something like `self.drop_prob = drop_prob`.

---
### LSTM Inputs/Outputs

You can create a basic [LSTM layer](https://pytorch.org/docs/stable/nn.html#lstm) as follows

```python
self.lstm = nn.LSTM(input_size, n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
```

where `input_size` is the number of characters this cell expects to see as sequential input, and `n_hidden` is the number of units in the hidden layers in the cell. And we can add dropout by adding a dropout parameter with a specified probability; this will automatically add dropout to the inputs or outputs. Finally, in the `forward` function, we can stack up the LSTM cells into layers using `.view`. With this, you pass in a list of cells and it will send the output of one cell into the next cell.

We also need to create an initial hidden state of all zeros. This is done like so

```python
self.init_hidden()
```


In [13]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


In [14]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=612, n_layers=4,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {c: i for i, c in self.int2char.items()}
        
        ## TODO: define the LSTM
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, dropout=drop_prob, batch_first=True)
        ## TODO: define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## TODO: define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars))
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## TODO: Get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        
        ## TODO: pass through a dropout layer
        out = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden)
        
        ## TODO: put x through the fully-connected layer
        out = self.fc(out)
        
       
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden
        

## Time to train

The train function gives us the ability to set the number of epochs, the learning rate, and other parameters.

Below we're using an Adam optimizer and cross entropy loss since we are looking at character class scores as output. We calculate the loss and perform backpropagation, as usual!

A couple of details about training: 
>* Within the batch loop, we detach the hidden state from its history; this time setting it equal to a new *tuple* variable because an LSTM has a hidden state that is a tuple of the hidden and cell states.
* We use [`clip_grad_norm_`](https://pytorch.org/docs/stable/_modules/torch/nn/utils/clip_grad.html) to help prevent exploding gradients.

In [15]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length))
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length))
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

## Instantiating the model

Now we can actually train the network. First we'll create the network itself, with some given hyperparameters. Then, define the mini-batches sizes, and start training!

In [27]:
# # define and print the net
n_hidden=512
n_layers=4

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(53, 512, num_layers=4, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=53, bias=True)
)


In [17]:
# Here we have loaded in a model that trained over 20 epochs `rnn_20_epoch.net`
# with open('rnn_20_epoch.net', 'rb') as f:
#     checkpoint = torch.load(f)
    
# loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
# loaded.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [28]:
batch_size = 64
seq_length = 160 #max length verses
n_epochs = 50 # start smaller if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)
# train(loaded, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/50... Step: 10... Loss: 3.1193... Val Loss: 3.0863
Epoch: 1/50... Step: 20... Loss: 2.9717... Val Loss: 2.9630
Epoch: 1/50... Step: 30... Loss: 2.9590... Val Loss: 2.9467
Epoch: 1/50... Step: 40... Loss: 2.9629... Val Loss: 2.9435
Epoch: 1/50... Step: 50... Loss: 2.9609... Val Loss: 2.9462
Epoch: 1/50... Step: 60... Loss: 2.9558... Val Loss: 2.9433
Epoch: 1/50... Step: 70... Loss: 2.9576... Val Loss: 2.9444
Epoch: 2/50... Step: 80... Loss: 2.9487... Val Loss: 2.9445
Epoch: 2/50... Step: 90... Loss: 2.9360... Val Loss: 2.9411
Epoch: 2/50... Step: 100... Loss: 2.9368... Val Loss: 2.9425
Epoch: 2/50... Step: 110... Loss: 2.9368... Val Loss: 2.9394
Epoch: 2/50... Step: 120... Loss: 2.9384... Val Loss: 2.9414
Epoch: 2/50... Step: 130... Loss: 2.9529... Val Loss: 2.9338
Epoch: 2/50... Step: 140... Loss: 2.9383... Val Loss: 2.9179
Epoch: 3/50... Step: 150... Loss: 2.8656... Val Loss: 2.8375
Epoch: 3/50... Step: 160... Loss: 2.7101... Val Loss: 2.7077
Epoch: 3/50... Step: 170... Loss:

In [29]:
model_name = 'rnn_50_epoch.net'

checkpoint = {'n_hidden': loaded.n_hidden,
              'n_layers': loaded.n_layers,
              'state_dict': loaded.state_dict(),
              'tokens': loaded.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

---
## Making Predictions

Now that the model is trained, we'll want to sample from it and make predictions about next characters! To sample, we pass in a character and have the network predict the next character. Then we take that character, pass it back in, and get another predicted character. Just keep doing this and you'll generate a bunch of text!

### A note on the `predict`  function

The output of our RNN is from a fully-connected layer and it outputs a **distribution of next-character scores**.

> To actually get the next character, we apply a softmax function, which gives us a *probability* distribution that we can then sample to predict the next character.

### Top K sampling

Our predictions come from a categorical probability distribution over all the possible characters. We can make the sample text and make it more reasonable to handle (with less variables) by only considering some $K$ most probable characters. This will prevent the network from giving us completely absurd characters while allowing it to introduce some noise and randomness into the sampled text. Read more about [topk, here](https://pytorch.org/docs/stable/torch.html#torch.topk).


In [30]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        # apply softmax to get p probabilities for the likely next character giving x
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        # considering the k most probable characters with topk method
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

### Priming and generating text 

Typically you'll want to prime the network so you can build up a hidden state. Otherwise the network will start out generating characters at random. In general the first bunch of characters will be a little rough since it hasn't built up a long history of characters to predict from.

In [31]:
def sample(net, size, prime='Il', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [38]:
file_name = "40_epoch_sample.txt"
file = open(file_name, "w")
# lyrics = sample(loaded, 1000, prime='wind', top_k=5)
lyrics = sample(net, 1000, prime='hello darkness my old friend', top_k=5)
file.write(lyrics)
file.close()
print(lyrics)

hello darkness my old friend, it's a hard, i have to go on fine, with a star she don't say no more. and where i've sound, so i can't get in. and if they'll be there. time is strange to the sun, and the show is a complicated, sit in the cold carss, since i'll be the one that i lived in mind, the floors are burning, the trees are breathin', i waited as shadows, this says are started to find. and the top in the whitken panding don't say goodbye
i said we say goodbye, i was let your feather comes so burning through the brain, why are you comin' from the ballors, i laugh to the sun, i can't let you down, i'd fight in your heart, to steal your stone of a man, a tall that seems, this land in the middle. of a town calling red and sold, we will never be so back about you, baby don't wanted to feel that death is not a love start, starsust in my love, as i was so sad and blue. i wanna rub-a-dub, dub lane, it might be love in the stars, but still i couldn't be there if you will be here to fight. y

## Loading a checkpoint